In [1]:
import os
import numpy as np

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.python.keras import backend as K
import imageio

from deepcell.applications import CellTrackingModel
from deepcell.applications import NuclearSegmentation
from deepcell_tracking import CellTracker

In [27]:
im = imageio.get_reader('../images/raw.gif')

mov = []

for frame in im:
    mov.append(frame)
mov = np.stack(mov, axis=0)
mov = np.expand_dims(mov, axis=-1)

print(images.shape)

(71, 512, 512, 1)


![Raw Gif](../images/raw.gif)

In [19]:
app = NuclearSegmentation()

W0413 19:50:10.628216 140150366025536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
print('Training Resolution:', app.model_mpp, 'microns per pixel')

Training Resolution: 0.65 microns per pixel


In [21]:
label_mov = app.predict(mov, image_mpp=0.65)
label_mov = np.expand_dims(label_mov, axis=-1)
label_mov.shape

(71, 512, 512, 1)

In [99]:
def plot(im,vmin,vmax):
    fig,ax = plt.subplots(figsize=(10,8))
    ax.imshow(im,cmap='jet',vmin=vmin,vmax=vmax)
    plt.axis('off')
    
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.close(fig)
    
    return image

imageio.mimsave('../images/labeled.gif', [plot(label_mov[i,...,0],
                                              np.min(label_mov),
                                              np.max(label_mov)) for i in range(label_mov.shape[0])])

![Labeled Gif](../images/labeled.gif)

In [28]:
from deepcell_toolbox.processing import normalize

mov = mov.astype('float32')
norm_mov = []
for frame in range(mov.shape[0]):
    f = mov[frame,...,0]
    norm_mov.append(normalize(f))
    
norm_mov = np.stack(norm_mov, axis=0)
norm_mov = np.expand_dims(norm_mov, axis=-1)

In [29]:
tracking_model = CellTrackingModel()
cell_tracker = CellTracker(norm_mov, label_mov, tracking_model, birth=0.99, death=0.99, division=0.9)

In [30]:
cell_tracker.track_cells()
cell_tracker.dump('results.trk')

/usr/local/lib/python3.6/dist-packages/deepcell_tracking/tracking.py:721: FutureWarning: The coordinates keyword argument to skimage.measure.regionprops is deprecated. All features are now computed in rc (row-column) coordinates. Please remove `coordinates="rc"` from all calls to regionprops before updating scikit-image.
  props = regionprops(np.squeeze(roi), coordinates='rc')[0]
/usr/local/lib/python3.6/dist-packages/deepcell_tracking/tracking.py:685: FutureWarning: The coordinates keyword argument to skimage.measure.regionprops is deprecated. All features are now computed in rc (row-column) coordinates. Please remove `coordinates="rc"` from all calls to regionprops before updating scikit-image.
  props = regionprops(np.squeeze(roi), coordinates='rc')


In [ ]:
# Open the track file
from deepcell_tracking.utils import load_trks

data = load_trks('results.trk')

lineage = data['lineages']  # linage information
X = data['X']  # raw X data
y = data['y']  # tracked y data

In [98]:
def plot(im1,im2,vmin,vmax):
    fig,ax = plt.subplots(1,2,figsize=(10,8))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[1].imshow(im2,cmap='jet',vmin=vmin,vmax=vmax)
    ax[1].axis('off')
    
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    
    return image

imageio.mimsave('../images/tracks.gif', [plot(X[i,...,0],
                                              y[i,...,0],
                                              np.min(y),
                                              np.max(y)) for i in range(label_mov.shape[0])])

![Tracked Gif](../images/tracks.gif)

In [15]:
# Save stacks
for i in range(X.shape[0]):
    if not os.path.isdir('nucleartracks'):
        os.mkdir('nucleartracks')
    
    new_image = X[i,...,0]
    imageio.imwrite('nucleartracks/image_%d.tiff' %i, new_image)

    label_image = y[i,...,0]
    imageio.imwrite('nucleartracks/label_%d.tiff' %i, label_image)